### Notebook Purpose

Perform quant and qual evaluation for fastmri results in ismrm paper

In [15]:
import os, sys
from os import listdir
from os.path import isfile, join
import numpy as np
import torch
import matplotlib.pyplot as plt

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.data_io import load_h5
from utils.evaluate import calc_metrics
from utils.transform import fft_2d, ifft_2d, root_sum_squares, \
                            reshape_complex_vals_to_adj_channels, \
                            reshape_adj_channels_to_complex_vals, \
                            crop_center

In [17]:
def load_gt(file_id):
    _, ksp_orig = load_h5(file_id)
    ksp_orig = torch.from_numpy(ksp_orig)
    return crop_center(root_sum_squares(ifft_2d(ksp_orig)), DIM, DIM)

def norm_im(im):
    ''' normalize img to [0,1] range '''
    return (im - im.min()) * (1. / im.max())

def plot_list(arr_list, clim=None):
    
    NUM_COLS = len(arr_list)
    title_list = ['ground-truth', 'dc', 'fm', 'diff']
    
    fig = plt.figure(figsize=(20,20))
    
    for idx in range(NUM_COLS):
        ax = fig.add_subplot(1,NUM_COLS,idx+1)
        ax.imshow(arr_list[idx], cmap='gray', clim=clim)
        ax.set_title(title_list[idx], fontsize=20)
        ax.axis('off')

### compare old dd+ (pytorch==1.5) vs new dd+ (pytorch==1.7)

from the baseline directory `/bmrNAS/people/dvv/out_fastmri/`...
- (1) `old_pytorch1.5/orig_alpha_search/` (n=11): using dd+ pt==1.5 before code re-work 
- (2) `old_pytorch1.5/sf0.1/` (n=5): using dd+ pt==1.5 after code re-work, then converted back
    - not sure if this is actually in pt==1.5 or if i just took pt==1.7 and used the deprecated fft/ifft functions in a hacky way. either way, i might have this stashed in a branch somewhere?
- (3) `new_pytorch1.7/sf0.1/` (n=5): using dd+ pt==1.7 sf0.1 w `fit.py`
- (4) `expmt_fm_loss/trials_best/` (n=11): using dd+ pt==1.7 sf=0.1 w `fit_feat_map_loss.py` instead of `fit.py`
    - i.e. looking at trial_id = `0000_10k`

##### results
- 1 > 4
- 2 > 1, but we only have five samples for 2
- 2, 3 are comparable
- ssim, psnr
    - (1)
        - n=5: 0.7884340675904544 31.730754973365197
        - n=11: 0.7569765714478369 30.87466793360071
    - (2) 0.796467652242989 32.034361010080524
    - (3) 0.793504546953909 31.93507930075753
    - (4)
        - n=5: 0.7647340218936469 30.56831878262691
        - n=11: 0.7399233683613828 29.98914482917978

# TODO
- ARJUN
    - implement vif, msssim --> send him code
    - get a decent dd+ model --> send him samples to run inference over
- try to re-create (3) by following the current codebase, i.e. same process for (4) but using `fit.py`
    - i.e. compare to files in `path_3` across n=5
- if that fails, look at old ipynb's and branches. try to re-create (3) first, but fall back to (2) if necessary
- once i re-create decent results, want to run this over a larger validation set. send to arjun

In [80]:
path_base = '/bmrNAS/people/dvv/out_fastmri/'
path_1 = path_base + 'old_pytorch1.5/orig_alpha_search/'
path_2 = path_base + 'old_pytorch1.5/sf0.1/'
path_3 = path_base + 'new_pytorch1.7/sf0.1/'
path_4 = path_base + '/expmt_fm_loss/trials_best/'

file_id_list = ['1000273', '1000325', '1000464', \
                '1000537', '1000818', '1001140', '1001219', \
                '1001338', '1001598', '1001533', '1001798']
# file_id_list = ['1000273', '1000325', '1000464', '1000537', '1000818']

In [50]:
DIM = 320
nf = len(file_id_list)

s_o_list, p_o_list, s_n_list, p_n_list = np.empty(nf), np.empty(nf), np.empty(nf), np.empty(nf)

for idx, file_id in enumerate(file_id_list):
    
    img_gt = load_gt(file_id)

#     img_old = np.load('{}{}_iter10000_alpha0.npy'.format(path_1, file_id))
    img_old = np.load('{}{}_dc.npy'.format(path_3, file_id))
#     img_new = np.load('{}0000_10k_{}_dc.npy'.format(path_4, file_id))

    img_new = XXXXXXXXXXXXXXXXXXXXXXXXXXX

    _, _, s_o_list[idx], p_o_list[idx] = calc_metrics(img_old, np.array(img_gt))
    _, _, s_n_list[idx], p_n_list[idx] = calc_metrics(img_new, np.array(img_gt))

    img_diff = np.abs(norm_im(img_old) - norm_im(img_new))

#     plot_list([img_gt, img_old, img_new, img_diff])

print(s_o_list.mean(), p_o_list.mean()) 
print(s_n_list.mean(), p_n_list.mean())

0.7569765714478369 30.87466793360071
0.7399233683613828 29.98914482917978


### load arjun results

In [52]:
import pandas as pd

In [84]:
# csv1 trained over more samples. use for slightly better results
# csv1 = '/bmrNAS/people/arjun/results/ss_recon/fastMRI_knee_mc/baseline_4x_dev/full/test_results/metrics.csv'
csv2 = '/bmrNAS/people/arjun/results/ss_recon/fastMRI_knee_mc/baseline_4x_dev/20_percent/test_results/metrics.csv'
df = pd.read_csv(csv2)

file_id_list_str = ['file' + str(f) for f in file_id_list]
df_ = df.loc[df['scan_name'].isin(file_id_list_str)]
df_ = df_[df_['rescaled']==True]


ssim_str = 'ssim_old' # use 'ssim (Wang)' for slightly better results
print(df_[ssim_str].mean())

df_

0.7458230879043853


,Unnamed: 0,scan_name,l1,l2,psnr,ssim_old,ssim (Wang),nrmse,l1_scan,l2_scan,...,ssim (Wang)_scan,nrmse_scan,Method,Acceleration,dataset,Noise Level,weights,rescaled,Criterion Name,Criterion Val
28,28,file1000273,0.000003,0.000003,30.475289,0.759324,0.770640,0.139624,0.000003,0.000003,...,0.878223,0.125549,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
37,37,file1000325,0.000004,0.000004,29.009453,0.711255,0.728130,0.220894,0.000004,0.000004,...,0.875412,0.214484,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
44,44,file1000464,0.000015,0.000020,28.481516,0.805303,0.820273,0.117621,0.000015,0.000022,...,0.886253,0.127346,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
50,50,file1000537,0.000007,0.000009,30.368369,0.819220,0.825489,0.113998,0.000007,0.000009,...,0.918550,0.119703,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
71,71,file1000818,0.000003,0.000003,28.208970,0.689889,0.697528,0.209001,0.000003,0.000003,...,0.847298,0.141234,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
98,98,file1001140,0.000004,0.000005,28.555031,0.705588,0.704900,0.250614,0.000004,0.000005,...,0.911234,0.175265,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
110,110,file1001219,0.000006,0.000007,30.388953,0.795738,0.802422,0.127796,0.000006,0.000008,...,0.920759,0.104399,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
117,117,file1001338,0.000003,0.000004,30.066637,0.757955,0.766777,0.155558,0.000003,0.000004,...,0.884056,0.141807,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
131,131,file1001533,0.000004,0.000004,27.851144,0.703727,0.714618,0.216354,0.000004,0.000005,...,0.864104,0.214195,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006
135,135,file1001598,0.000005,0.000005,29.942862,0.771781,0.778072,0.142423,0.000005,0.000006,...,0.906754,0.101779,GeneralizedUnrolledCNN,4,fastMRI_knee_multicoil_v0.0.1_test,0,model_0058057.pth,True,val_l1,0.000006


In [91]:
df = pd.read_csv(csv1)
dff = df[df['rescaled']==True]
dff['psnr'].mean(), dff['ssim (Wang)'].mean()

(29.76801405082239, 0.7756780905932461)